In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from scipy import stats
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
def changeDataOfPrice(row):
  priceRange = row[20]
  if(priceRange == 0 or priceRange == 1):
    return 0
  else:
    return 1

make 2 classes

In [ ]:
df['price_range'] = df.apply(changeDataOfPrice,axis='columns')
df['price_range'].value_counts()

0    1000
1    1000
Name: price_range, dtype: int64

# Forward and backward selection

In [ ]:
df_copy = df.copy()

In [ ]:
y = df_copy.pop('price_range')
X = df_copy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_train.shape)

(1600, 20)
(1600,)
(400, 20)
(1600,)


scale the data

In [ ]:
scaler_1 = StandardScaler()

In [ ]:
fitter_1 = scaler_1.fit(X_train)

In [ ]:
X_train = fitter_1.transform(X_train)
X_test = fitter_1.transform(X_test)

In [ ]:
X_train = pd.DataFrame(data=X_train, columns=X.columns)
X_test = pd.DataFrame(data=X_test, columns=X.columns)

In [ ]:
def LR(X_train, y_train, X_test, y_test, v):
    LR = LogisticRegression()
    LR = LR.fit(X_train, y_train)
    yhat = LR.predict(X_test)
    acc = accuracy_score(y_test, yhat)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, yhat)
    acc_auc = metrics.auc(fpr, tpr)  
    if v == 1:
        return (acc_auc)
      #predict test data we need model
    elif v == 2:
        return LR
    #all features ,pca
    elif v == 3:
      message = f"Accuracy of sklearn's Logistic Regression Classifier with forward selectopn: {acc}, acc_auc: {acc_auc}"
      return (message, yhat)
    elif v == 4 : 
      message = f"Accuracy of sklearn's Logistic Regression Classifier with backward selectopn: {acc}, acc_auc: {acc_auc}"
      return (message, yhat)  

**forward** **selection**

In [ ]:
def forward(X_train, y_train, X_test, y_test, best_cols, all_cols):
    init_acc_auc = 0.50
    for col in all_cols:
        best_cols.append(col)
        X_train_f = pd.DataFrame(data=X_train, columns=best_cols)
        X_test_f = pd.DataFrame(data=X_test, columns=best_cols)
        acc_auc = LR(X_train_f, y_train, X_test_f, y_test, v=1)
        if acc_auc > init_acc_auc:
            init_acc_auc = acc_auc
        else:
            best_cols.pop()
    return(f'result forward selection=> best columns: {best_cols} with auc: {init_acc_auc}', best_cols)

In [ ]:
best_cols, all_cols = [], X.columns.to_list()
forward_col, best_cols_f = forward(X_train, y_train, X_test, y_test, best_cols, all_cols)
print(forward_col)

result forward selection=> best columns: ['battery_power', 'blue', 'clock_speed', 'n_cores', 'px_width', 'ram'] with auc: 0.9725465090005272


In [ ]:
X_train_f = pd.DataFrame(data=X_train, columns=best_cols_f)
X_test_f = pd.DataFrame(data=X_test, columns=best_cols_f)
message, yhat = LR(X_train_f, y_train, X_test_f, y_test, 3)
print(message)
print(classification_report(y_test, yhat))

Accuracy of sklearn's Logistic Regression Classifier with forward selectopn: 0.9725, acc_auc: 0.9725465090005272
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       187
           1       0.98      0.97      0.97       213

    accuracy                           0.97       400
   macro avg       0.97      0.97      0.97       400
weighted avg       0.97      0.97      0.97       400



**backward selection**

In [ ]:
def backward(X_train, y_train, X_test, y_test, all_cols):
    init_acc_auc = LR(X_train, y_train, X_test, y_test, 1)
    for i in range(len(all_cols)-1):
        col = all_cols.pop(0)
        X_train_b = pd.DataFrame(data=X_train, columns=all_cols)
        X_test_b = pd.DataFrame(data=X_test, columns=all_cols)
        acc_auc = LR(X_train_b, y_train, X_test_b, y_test, 1)
        if acc_auc > init_acc_auc:
            init_acc_auc = acc_auc
        else:
            all_cols.append(col)
    return(f'result backward selection=> best columns: {all_cols} with auc: {init_acc_auc}', all_cols)

In [ ]:
all_cols = X.columns.to_list()
backward_col, best_cols_b = backward(X_train, y_train, X_test, y_test, all_cols)
print(backward_col)

result backward selection=> best columns: ['wifi', 'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g'] with auc: 0.986957394993849


In [ ]:
X_train_b = pd.DataFrame(data=X_train, columns=best_cols_b)
X_test_b = pd.DataFrame(data=X_test, columns=best_cols_b)
message, yhat = LR(X_train_b, y_train, X_test_b, y_test, 4)
print(message)
print(classification_report(y_test, yhat))

Accuracy of sklearn's Logistic Regression Classifier with backward selectopn: 0.9875, acc_auc: 0.986957394993849
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       187
           1       0.98      1.00      0.99       213

    accuracy                           0.99       400
   macro avg       0.99      0.99      0.99       400
weighted avg       0.99      0.99      0.99       400



with out forward or backward selection

In [ ]:
x_train = df.drop('price_range',axis=1).values
y_train = df['price_range'].values

x_test = df.drop('price_range',axis=1).values
y_test = df['price_range'].values

model = LogisticRegression()
model.fit(x_train,y_train)
y_pred1= model.predict(x_test)

print(model.score(x_test,y_test))
print(classification_report(y_test, y_pred1))

0.893
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      1000
           1       0.88      0.91      0.89      1000

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# using PCA

In [ ]:
dfp  = pd.read_csv('train.csv')
x = dfp.drop('price_range',axis=1).values
y = dfp['price_range'].values
print(x.shape)
print(y.shape)

(2000, 20)
(2000,)


In [ ]:
  pca = PCA(6)
  xPCA = pca.fit_transform(x)
  print(xPCA.shape)

  XPCA_train,XPCA_test,y_train,y_test = train_test_split(xPCA,y,test_size=0.2, random_state=30)
  model = LogisticRegression()
  model.fit(XPCA_train,y_train)
  y_pred2= model.predict(XPCA_test)
  print("score of the model is :")
  print(model.score(XPCA_test,y_test))
  print(classification_report(y_test, y_pred2))


(2000, 6)
score of the model is :
0.6325
              precision    recall  f1-score   support

           0       0.55      1.00      0.71        99
           1       0.89      0.16      0.28        97
           2       1.00      0.25      0.40        88
           3       0.64      1.00      0.78       116

    accuracy                           0.63       400
   macro avg       0.77      0.60      0.54       400
weighted avg       0.76      0.63      0.56       400



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# feature engineering

In [3]:
dfp  = pd.read_csv('train.csv')
x = dfp.drop('price_range',axis=1).values
y = dfp['price_range'].values
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=40)

**binning the battery power**

In [4]:
dfp.battery_power.describe()

count    2000.000000
mean     1238.518500
std       439.418206
min       501.000000
25%       851.750000
50%      1226.000000
75%      1615.250000
max      1998.000000
Name: battery_power, dtype: float64

In [5]:
def binningBattery(x):
    if x >= 1600:
        return 0.9
    elif 1200 <= x < 1600:
        return 0.7
    elif 800 <= x < 1200 :
        return 0.5
    else:
        return 0.3

In [6]:
dfp.battery_power = dfp.battery_power.apply(binningBattery)

In [7]:
dfp.battery_power.describe()

count    2000.000000
mean        0.613200
std         0.218104
min         0.300000
25%         0.500000
50%         0.700000
75%         0.900000
max         0.900000
Name: battery_power, dtype: float64

svm

In [14]:
from sklearn import svm
y_t = np.array(dfp['price_range'])
X_t = dfp
X_t = dfp.drop(['price_range'],axis=1)
X_t = np.array(X_t)
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)

In [17]:
clf = svm.SVC()
clf.fit(X_train, Y_train)
clf.predict(X_test)
scoretrain = clf.score(X_train,Y_train)
scoretest  = clf.score(X_test,Y_test)
print("train score and test score")
print(scoretrain)
print(scoretest)

train score and test score
0.806875
0.805


**one hot encoding**

In [18]:
dfp = pd.read_csv('train.csv')

In [19]:
pd.set_option('display.max_columns',None)
dfp.head(10)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1
5,1859,0,0.5,1,3,0,22,0.7,164,1,7,1004,1654,1067,17,1,10,1,0,0,1
6,1821,0,1.7,0,4,1,10,0.8,139,8,10,381,1018,3220,13,8,18,1,0,1,3
7,1954,0,0.5,1,0,0,24,0.8,187,4,0,512,1149,700,16,3,5,1,1,1,0
8,1445,1,0.5,0,0,0,53,0.7,174,7,14,386,836,1099,17,1,20,1,0,0,0
9,509,1,0.6,1,2,1,9,0.1,93,5,15,1137,1224,513,19,10,12,1,0,0,0


In [20]:
dfp.nunique()

battery_power    1094
blue                2
clock_speed        26
dual_sim            2
fc                 20
four_g              2
int_memory         63
m_dep              10
mobile_wt         121
n_cores             8
pc                 21
px_height        1137
px_width         1109
ram              1562
sc_h               15
sc_w               19
talk_time          19
three_g             2
touch_screen        2
wifi                2
price_range         4
dtype: int64

svm

In [21]:
from sklearn import svm
y_t = np.array(dfp['price_range'])
X_t = dfp
X_t = dfp.drop(['price_range'],axis=1)
X_t = np.array(X_t)
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)

clf = svm.SVC()
clf.fit(X_train, Y_train)
clf.predict(X_test)
scoretrain = clf.score(X_train,Y_train)
scoretest  = clf.score(X_test,Y_test)
print("train score and test score")
print(scoretrain)
print(scoretest)

train score and test score
0.95375
0.965


**log transform**

In [40]:
dfp = pd.read_csv('train.csv')

In [41]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, subplot_titles=("Histogram of clock_speed before transformation", "Histogram of Sulphates After transformation"))
                    

fig.append_trace(go.Histogram(x = dfp['clock_speed']), row = 1, col = 1)
fig.append_trace(go.Histogram(x = 1/(np.log(dfp['clock_speed'])), nbinsx=40), row = 1, col = 2)
fig.update_layout(height=600, width= np.inf, title_text="Distribution of clock_speed before and after 1/log(y) transformation", showlegend=False, title_x = 0.5)
fig.show()

In [43]:
pt = PowerTransformer(method = 'box-cox')
pt.fit(dfp['clock_speed'].values.reshape(-1, 1))
dfp = pt.transform(dfp['clock_speed'].values.reshape(-1, 1))
print(f"Value of Lambda : {pt.lambdas_}")

Value of Lambda : [0.42436989]


In [42]:
from sklearn import svm
y_t = np.array(dfp['price_range'])
X_t = dfp
X_t = dfp.drop(['price_range'],axis=1)
X_t = np.array(X_t)
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)

clf = svm.SVC()
clf.fit(X_train, Y_train)
clf.predict(X_test)
scoretrain = clf.score(X_train,Y_train)
scoretest  = clf.score(X_test,Y_test)
print("train score and test score")
print(scoretrain)
print(scoretest)

train score and test score
0.95375
0.965


In [25]:
dfp.clock_speed

0       2.2
1       0.5
2       0.5
3       2.5
4       1.2
       ... 
1995    0.5
1996    2.6
1997    0.9
1998    0.9
1999    2.0
Name: clock_speed, Length: 2000, dtype: float64

# **add new feature **

In [29]:
dfp = pd.read_csv('train.csv')

In [30]:
dfp.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


In [31]:
dfp["Volume"] = dfp['m_dep']*dfp['sc_h']*dfp['sc_w']

In [32]:
dfp.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range,Volume
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,9.916500,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000,39.793850
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,6.064315,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314,50.200476
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,0.000000,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,5.000000,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000,6.375000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,10.000000,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000,19.800000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,15.000000,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000,54.000000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,20.000000,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000,342.000000


svm

In [33]:
from sklearn import svm
y_t = np.array(dfp['price_range'])
X_t = dfp
X_t = dfp.drop(['price_range'],axis=1)
X_t = np.array(X_t)
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)

clf = svm.SVC()
clf.fit(X_train, Y_train)
clf.predict(X_test)
scoretrain = clf.score(X_train,Y_train)
scoretest  = clf.score(X_test,Y_test)
print("train score and test score")
print(scoretrain)
print(scoretest)

train score and test score
0.95375
0.965


# SVM OF ALL

In [50]:
dfp  = pd.read_csv('train.csv')

In [51]:
from sklearn import svm
y_t = np.array(dfp['price_range'])
X_t = dfp
X_t = dfp.drop(['price_range'],axis=1)
X_t = np.array(X_t)
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)

clf = svm.SVC()
clf.fit(X_train, Y_train)
clf.predict(X_test)
scoretrain = clf.score(X_train,Y_train)
scoretest  = clf.score(X_test,Y_test)
print("train score and test score")
print(scoretrain)
print(scoretest)

train score and test score
0.95375
0.965
